## Теорема Байеса
$ P(A|B) = \frac{P(B|A) P(A)}{P(B)} $, где

$P(B|A)$ — вероятность наступления события B при истинности гипотезы A;
$P(A)$ — априорная вероятность гипотезы A;
$P(B)$ —  полная вероятность наступления события B;
$P(A|B$ —  априорная вероятность гипотезы A.


## Наивный байесовский классификатор
$P(C|D) = \frac{P(D|C) P(C)}{P(D)}$, где
$P(C|D)$ — вероятность что документ $D$ принадлежит классу $C$;
$P(D|C)$ — вероятность встретить документ $D$ среди всех документов класса $C$;
$P(C)$ — безусловная вероятность встретить документ класса $C$ в корпусе документов (упр. список всех слов);
$P(D)$ — безусловная вероятность документа $D$ в корпусе документов.

Вероятность $P(D|C)$ можно найти с помощью произведения вероятностей всех слов входящих в документ:
$P(D|C) \approx \prod_{i=1}^n P(w_{i}|C)$, где
$n$ — количество слов;
$w_{i}$ — слово $w$ с индексом $i$;
$P(w_{i}|C)$ — оценка вероятности встретить слово $w_{i}$ в классе $С$. Существуют различные методы оценки. Например, multinomial bayes model:
$P(w_{i}|C) = \frac{W_{ic}}{\sum_{i \in V} W_{ic}}$, где
$W_{ic}$ — количество вхождения $i$-го слова в документы класса $C$
$V$ — словарь корпуса документов (список всех уникальных слов)
Простыми словами:
- числитель описывает сколько раз слово встречается в документах класса (включая повторы),
- знаменатель – это суммарное количество слов во всех документах этого класса.

